# Autoencoder for Noise Removal on Fashion MNIST
This notebook demonstrates how a convolutional autoencoder can be used to remove noise from the Fashion MNIST dataset. We add noise to the images, then train a denoising autoencoder to reconstruct the original images. We also evaluate the model's performance using PSNR (Peak Signal-to-Noise Ratio) and SSIM (Structural Similarity Index).

---



### **1. Setup & Importing Libraries**
First, we import all the necessary libraries for the project.



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.ndimage import gaussian_filter
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd


2. Load & Prepare the Fashion MNIST Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Split training set into train and validation sets
X_train, X_valid, X_test = X_train[5000:] / 255.0, X_train[:5000] / 255.0, X_test / 255.0
y_train, y_valid = y_train[5000:], y_train[:5000]

# Reshape to add channel dimension for CNN input
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]


#3. Add Noise to the Images

In [ ]:
noise_factor = 0.2
X_train_noisy = np.clip(X_train + noise_factor * np.random.rand(*X_train.shape), 0, 1)
X_valid_noisy = np.clip(X_valid + noise_factor * np.random.rand(*X_valid.shape), 0, 1)
X_test_noisy = np.clip(X_test + noise_factor * np.random.rand(*X_test.shape), 0, 1)


4. Visualize Noisy Images

In [ ]:
def plot_noise_images(X_clean, X_noisy, title):
    plt.figure(figsize=(14, 3))
    plt.suptitle(title, fontsize=16, y=1.05)
    num_images = min(10, len(X_clean))  # Display up to 10 images
    for i in range(num_images):
        plt.subplot(2, 10, i + 1)
        plt.imshow(X_clean[i], cmap='binary')
        plt.title('Original', fontsize=8)
        plt.axis('off')

        plt.subplot(2, 10, i + 11)
        plt.imshow(X_noisy[i], cmap='binary')
        plt.title('Noisy', fontsize=8)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

plot_noise_images(X_train, X_train_noisy, "Original and Noisy Training Images")


# 5. Define the Denoising Autoencoder Model

In [ ]:
class DenoiseAutoEncoder(keras.Model):
    def __init__(self, input_shape, **kwargs):
        super().__init__(**kwargs)
        self.encoder_ = keras.Sequential([
            keras.layers.InputLayer(shape=input_shape),
            keras.layers.Conv2D(16, 3, padding='same', strides=2, activation='relu'),
            keras.layers.Conv2D(8, 3, padding='same', strides=2, activation='relu')
        ])
        self.decoder_ = keras.Sequential([
            keras.layers.Conv2DTranspose(8, 3, padding='same', strides=2, activation='relu'),
            keras.layers.Conv2DTranspose(16, 3, padding='same', strides=2, activation='relu'),
            keras.layers.Conv2D(1, 3, padding='same', strides=1, activation='sigmoid')
        ])

    def call(self, x):
        latent_space = self.encoder_(x)
        output_ = self.decoder_(latent_space)
        return output_


# 6. Train the Denoising Autoencoder

In [ ]:
# Initialize the model
image_shape = X_train_noisy.shape[1:]
model = DenoiseAutoEncoder(image_shape)

# Compile the model
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Train the model
history = model.fit(X_train_noisy, X_train, epochs=30, shuffle=True, validation_data=(X_valid_noisy, X_valid), verbose=1)

# Plot training history
pd.DataFrame(history.history).plot()
plt.grid()
plt.show()


# 7. Reconstruct Images Before and After Training

In [ ]:
def model_reconstruct_images(model, X_noisy, model_status='before'):
    x = X_noisy[:10]
    y = model(X_noisy[:10])
    plt.figure(figsize=(14, 3))
    plt.suptitle(f"{model_status.capitalize()} Model Reconstructed Images", fontsize=16, y=1.05)
    for i in range(len(x)):
        plt.subplot(2, 10, i + 1)
        plt.imshow(x[i], cmap='binary')
        plt.title('Noisy Input', fontsize=8)
        plt.axis('off')

        plt.subplot(2, 10, i + 11)
        plt.imshow(y[i], cmap='binary')
        plt.title('Reconstructed', fontsize=8)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Reconstruct before training
model_reconstruct_images(model, X_train_noisy, model_status='before')

# Reconstruct after training
model_reconstruct_images(model, X_train_noisy, model_status='after')


# 8. Apply Gaussian Blur to Test Set

In [ ]:
# Apply Gaussian blur to each image in the noisy test set
X_test_blurred = np.array([gaussian_filter(img.squeeze(), sigma=1) for img in X_test_noisy])

# Reshape back to (28, 28, 1)
X_test_blurred = X_test_blurred[..., np.newaxis]

# Visualize the blurred images
plot_noise_images(X_test_noisy, X_test_blurred, "Noisy Images and Gaussian Blurred Versions")


# 9. Latent Space Visualization with t-SNE

In [ ]:
# Visualize the latent space with t-SNE
X_val_noise_pred = model.encoder_.predict(X_valid_noisy).reshape(X_valid.shape[0], -1)
X_val_noise_tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200).fit_transform(X_val_noise_pred)

# Visualize the compressed latent space
def visualize_compressed(x_compressed, y, images, result='PCA_Results', class_names=None):
    plt.figure(figsize=(8, 6))
    cmap = plt.cm.tab10
    Z = (x_compressed - x_compressed.min()) / (x_compressed.max() - x_compressed.min())  # normalize
    scatter = plt.scatter(Z[:, 0], Z[:, 1], c=y, s=10, cmap=cmap)
    image_positions = np.array([[1., 1.]])
    for index, position in enumerate(Z):
        dist = ((position - image_positions) ** 2).sum(axis=1)
        if dist.min() > 0.02:  # avoid overlapping image previews
            image_positions = np.r_[image_positions, [position]]
            imagebox = mpl.offsetbox.AnnotationBbox(
                mpl.offsetbox.OffsetImage(images[index].reshape(28, 28), cmap="binary", zoom=0.6),
                position,
                bboxprops={"edgecolor": cmap(y[index]), "lw": 1}
            )
            plt.gca().add_artist(imagebox)

    if class_names:
        handles = [plt.Line2D([0], [0], marker='o', color='w', label=class_names[i],
                              markerfacecolor=cmap(i), markersize=6)
                   for i in np.unique(y)]
        plt.legend(handles=handles, loc='best', fontsize=10)

    plt.axis("off")
    plt.title(f"{result}: 2D Latent Space with Class Labels and Sample Images", fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_compressed(X_val_noise_tsne, y_valid, X_valid, result='Autoencoder_Results', class_names=['Cluster ' + str(i) for i in range(10)])


# 10. Evaluation: PSNR and SSIM

In [ ]:
# Predict denoised test images
denoised = model.predict(X_test_noisy, verbose=0)

# Evaluate PSNR
avg_psnr = np.mean([psnr(X_test[i].squeeze(), denoised[i].squeeze(), data_range=1.0)
                    for i in range(len(X_test))])
print("Average PSNR > 30 (preferable and indicates excellent denoising):", avg_psnr)

# Evaluate SSIM
avg_ssim = np.mean([ssim(X_test[i].squeeze(), denoised[i].squeeze(), data_range=1.0)
                    for i in range(len(X_test))])
print("Average SSIM ~ 1 (Closer to 1.0 is ideal, reflects structural similarity):", avg_ssim)


## Conclusion

Our denoising autoencoder was able to significantly clean noisy images from the Fashion MNIST dataset.

- **Average PSNR**: ~25.7 — quite good but leaves room for optimization (ideal is >30).
- **Average SSIM**: ~0.90 — indicating good structural preservation.

### Future Improvements:
- Use deeper architectures or skip connections (e.g., U-Net).
- Try different types of noise (salt & pepper, speckle).
- Use adversarial training for sharper outputs (e.g., Denoising GAN).

This setup demonstrates a practical application of unsupervised learning to clean noisy data and recover structured input — a valuable tool in image processing pipelines.
